In [1]:
%load_ext dotenv
%load_ext google.cloud.bigquery
%dotenv ../.env
%cd /home/m087494/PycharmProjects/CAPQ-TCGA

/home/m087494/PycharmProjects/CAPQ-TCGA


In [2]:
import os
import logging
from utils.utils import create_slides_metadata, get_data
GCP_PROJECT_ID = os.environ.get('GCP_PROJECT_ID')
logging.basicConfig(level=getattr(logging, 'INFO'), format='%(message)s')

In [3]:
# Set Dynamic Variables
data_dir='data'
workspace_dir = 'workspace'

In [4]:
# Create Working spaces
for directory in [data_dir, workspace_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

## TODO: Update SQL code to get the slides I want

In [5]:
%%bigquery bq_results_df --project=$GCP_PROJECT_ID
SELECT SeriesInstanceUID AS digital_slide_id,
StudyInstanceUID AS case_id,
ContainerIdentifier AS physical_slide_id,
PatientID AS patient_id,
TotalPixelMatrixColumns AS width,
TotalPixelMatrixRows AS height,
collection_id,
crdc_instance_uuid,
gcs_url,
gcs_bucket,
CAST(SharedFunctionalGroupsSequence[SAFE_OFFSET(0)].
      PixelMeasuresSequence[SAFE_OFFSET(0)].
      PixelSpacing[SAFE_OFFSET(0)] AS FLOAT64) AS pixel_spacing,
CASE TransferSyntaxUID
  WHEN '1.2.840.10008.1.2.4.50' THEN 'jpeg'
  WHEN '1.2.840.10008.1.2.4.91' THEN 'jpeg2000'
  ELSE 'other'
END AS compression,

FROM `bigquery-public-data.idc_v18.dicom_all`
WHERE
  Modality = 'SM'
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

Identify data files to use

In [6]:
slides_metadata = create_slides_metadata(bq_results_df)

Download the DICOM files locally

In [23]:
for x, j in slides_metadata.iterrows():
    gcs_url = j['levels'][0]['gcs_url']
    gs, _, bucket = gcs_url.split('/')[:3]
    blob = '/'.join(gcs_url.split('/')[3:])
    get_data(blobs=[blob], bucket_name=bucket, outname=os.path.join(data_dir, blob))
    

Forbidden: 403 GET https://storage.googleapis.com/storage/v1/b/public-datasets-idc?projection=noAcl&prettyPrint=false: Steven.N.Hart@gmail.com does not have storage.buckets.get access to the Google Cloud Storage bucket. Permission 'storage.buckets.get' denied on resource (or it may not exist).

['gs:', '', 'public-datasets-idc']